# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.120665e+02     1.667883e+00
 * time: 0.7857649326324463
     1     1.038995e+01     1.004390e+00
 * time: 2.1748270988464355
     2    -1.208186e+01     1.031574e+00
 * time: 2.8572731018066406
     3    -3.436596e+01     8.457768e-01
 * time: 3.8737571239471436
     4    -4.793199e+01     6.147369e-01
 * time: 4.91091513633728
     5    -5.727178e+01     2.590683e-01
 * time: 5.9081950187683105
     6    -5.997830e+01     1.834429e-01
 * time: 6.581967115402222
     7    -6.102429e+01     5.184555e-02
 * time: 7.253514051437378
     8    -6.144735e+01     6.902266e-02
 * time: 7.941586017608643
     9    -6.169507e+01     4.531897e-02
 * time: 8.61252212524414
    10    -6.188497e+01     2.374962e-02
 * time: 9.284495115280151
    11    -6.201885e+01     2.164986e-02
 * time: 9.95748496055603
    12    -6.208931e+01     1.636555e-02
 * time: 10.630265951156616
    13    -6.214321e+01     1.401187e-02
 * time: 11.308933973312378
   

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671068
    AtomicLocal         -18.8557673
    AtomicNonlocal      14.8522641
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485370 
    Xc                  -19.3336819

    total               -62.261666462259
